# Спрос на рынке жилья (за счет собственных средств)

In [1]:
from os import path, getcwd
from IPython.display import Markdown as md
import pandas as pd
import numpy as np
import sqlalchemy as sa
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

%run ../COMMON/common.ipynb # загрузка общих функций и констант

In [2]:
conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных

pdf=(pd.read_sql(make_SELECT_YEAR_string(['GDP_x', 'GDP_Iq','CPIAv','KeyRate','Unmpl_s']), con=conWork)
     .pipe(make_frame)
     .pipe(scale, list_fields=['GDP_x', 'GDP_Iq'], multiplier=100)
     .pipe(add, 'Unmpl_s', -100))
pdf

,CPIAv,GDP_Iq,GDP_x,KeyRate,Unmpl_s
date,,,,,
2004,1.108812,107.175949,1702.719086,0.135200,-99.922851
2005,1.127055,106.376187,2160.976549,0.129800,-99.928468
2006,1.096830,108.153432,2691.720138,0.088942,-99.929378
2007,1.089881,108.535080,3324.751323,0.075126,-99.939706
2008,1.141048,105.247954,4127.684919,0.081291,-99.937588
2009,1.116977,92.179115,3880.721857,0.113100,-99.917478
2010,1.068508,104.503726,4630.854119,0.080300,-99.926555
2011,1.084634,104.264177,6011.400084,0.081200,-99.935044
2012,1.050667,104.024086,6810.344963,0.080700,-99.945413
